# <b>CONVERSOR DE NOMES LÓGICOS PARA FÍSICOS</b>

## <b>1. Instalação e importação de bibliotecas</b>

In [1]:
# Descomente a linha a seguir apenas no primeiro uso. A lib "openpyxl" é necessária para o Pandas ler arquivos do Excel
#!pip install openpyxl

In [2]:
import pandas as pd
from unidecode import unidecode
from typing import Union, List

## <b>2. Importação do Glossário de termos</b>

In [3]:
# Abre o glossário no formato Excel (altere a localização do glossário, se necessário)
df_glossario = pd.read_excel('Glossário de termos.xlsx')

## <b>3. Função de conversão de nome lógico</b>

In [4]:
def converter_para_nome_fisico(glossario: pd.DataFrame, nomes_logicos: Union[str, List[str]]) -> pd.DataFrame:
    # Conjunto de palavras a serem removidas do nome lógico
    PALAVRAS_REMOVER = {'o', 'as', 'os', 'um', 'uma', 'uns', 'umas', 'de', 'do', 'da', 'dos', 'das', 'em', 'no', 'na', 'nos', 'nas', 'por', 'para', 'com', 'sem', 'sobre', 'entre', 'e', 'ou', 'mas', 'se', 'que'}
    
    # Dicionário de substituições baseado no glossário
    substituicoes = dict(zip(glossario['NOME'].str.strip(), glossario['ABREVIAÇÃO'].str.strip()))

    def normalizar(texto: str) -> str:
        # Remove acentos, converte para maiúsculas e preserva aspas
        return ''.join(c if c in '"' else unidecode(c) for c in texto).upper()

    def tratar_nome_logico(nome_logico_normalizado: str) -> List[str]:
        palavras = nome_logico_normalizado.split()
        # Processa as palavras, unindo 'A' com a próxima palavra e removendo palavras desnecessárias
        return [f"{p} {palavras[i+1]}" if p == 'A' and i < len(palavras) - 1 else p for i, p in enumerate(palavras) if p.lower() not in PALAVRAS_REMOVER]

    def converter(nome_logico: str) -> str:
        nome_logico_normalizado = normalizar(nome_logico)
        palavras = tratar_nome_logico(nome_logico_normalizado)
        
        # Verifica se o nome completo está no glossário
        if nome_logico_normalizado in substituicoes:
            return substituicoes[nome_logico_normalizado]
        
        # Verifica se todas as palavras estão no glossário
        if all(p in substituicoes for p in palavras):
            return '_'.join(substituicoes[p] for p in palavras)
        
        # Retorna '---' se não houver correspondência
        return '---'

    # Garante que nomes_logicos seja uma lista
    nomes_logicos = [nomes_logicos] if isinstance(nomes_logicos, str) else nomes_logicos
    
    # Aplica a conversão para cada nome lógico
    nomes_fisicos = [converter(nome) for nome in nomes_logicos]

    # Retorna um DataFrame com os resultados
    return pd.DataFrame({'Nome Lógico': nomes_logicos, 'Nome Físico': nomes_fisicos})

## <b>4. Lista de nomes lógicos a serem convertidos</b>

In [5]:
nomes_logicos = [
    'A granel',
    'Ponta "B"',
    'São Paulo Futebol Clube',
    'Agência Nacional de Saúde Suplementar',
    'Agência-Tronco',
    'Alívio de numerário',
    'AnALiSe De SeNsIbIlIdAdE',
    'Assistência técnica',
    'Código do Curso',
    'Nome do Curso',
    'Texto do Curso'
]

## <b>5. Conversão para nomes físicos</b>

In [6]:
nomes_fisicos = converter_para_nome_fisico(df_glossario, nomes_logicos)

In [7]:
nomes_fisicos

,Nome Lógico,Nome Físico
0,A granel,GRAN
1,"Ponta ""B""",PNTB
2,São Paulo Futebol Clube,---
3,Agência Nacional de Saúde Suplementar,ANS
4,Agência-Tronco,AGTC
5,Alívio de numerário,ALNM
6,AnALiSe De SeNsIbIlIdAdE,ANLS
7,Assistência técnica,ATCN
8,Código do Curso,CD_CSO
9,Nome do Curso,NM_CSO
